In [1]:
import os
import time
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
from tqdm import tqdm
import seaborn as sns
import datetime
import time
import matplotlib.pyplot as plt
import statsmodels.stats as stat
import sys
import scipy.stats as stat
import sympy as sy
from sympy.stats import Normal, cdf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold,KFold
import sklearn
import joblib
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import lightgbm as lgb
import sklearn as sk
!pip install ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
#import pickle5 as pickle
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
tqdm.pandas(desc="my bars:")
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 115 kB/s 
     |████████████████████████████████| 8.8 MB 35.9 MB/s 
     |████████████████████████████████| 4.1 MB 47.5 MB/s 
     |████████████████████████████████| 466 kB 56.4 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Uninstalling grpcio-1.47.0:
      Successfully uninstalled grpcio-1.47.0


In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
feature_train = pd.read_csv("/content/drive/MyDrive/Huatai_ml_model/feature_train.csv",index_col=0)
feature_test = pd.read_csv("/content/drive/MyDrive/Huatai_ml_model/feature_test.csv",index_col=0)
X_train = feature_train.drop(['time','code','option_code','type','maturity_date','f_close_1min','f_ret_1min','f_ret_direction'],axis=1)
y_train = feature_train['f_ret_direction']
X_test = feature_test.drop(['time','code','option_code','type','maturity_date','f_close_1min','f_ret_1min','f_ret_direction'],axis=1)
y_test = feature_test['f_ret_direction']
#lstm dataset
stock_feature_train_list = []
stock_feature_test_list = []
code_list = feature_train.code.unique()
for code in code_list:
    stock_feature_train_list.append(feature_train[feature_train.code==code])
    stock_feature_test_list.append(feature_test[feature_test.code==code])

In [4]:
def batch_data_num(feature_list,sequence_len,batchsize):
    #return dataloader_list
    dataloader_list = []
    for features in feature_list:
        feature_tensors = []
        target_tensors = []
        if len(features)<sequence_len:
            continue
        for idx in range(0,len(features) - sequence_len):

            x_train = features.drop(['time','code','option_code','type','maturity_date','f_close_1min','f_ret_direction'],axis=1)
            y_train = features['f_close_1min']
            feature_tensors.append(x_train.iloc[idx:idx+sequence_len].values)
            target_tensors.append(y_train.iloc[idx])
        feature_tensors = torch.from_numpy(np.array(feature_tensors))
        target_tensors = torch.from_numpy(np.array(target_tensors))
        data = TensorDataset(feature_tensors,target_tensors)
        data_loader = torch.utils.data.DataLoader(data,shuffle=False,batch_size = batchsize)
        dataloader_list.append(data_loader)
    return dataloader_list

def batch_data_1(feature_list,sequence_len,batchsize,y_name):
    #return one dataloader
    feature_tensors = []
    target_tensors = []
    for features in feature_list:
        if len(features)<sequence_len:
            continue
        for idx in range(0,len(features) - sequence_len):
            x_train = features.drop(['time','code','option_code','type','maturity_date','f_close_1min','f_ret_1min','f_ret_direction'],axis=1)
            y_train = features[y_name]
            feature_tensors.append(x_train.iloc[idx:idx+sequence_len].values)
            target_tensors.append(y_train.iloc[idx])
    feature_tensors = torch.from_numpy(np.array(feature_tensors))
    target_tensors = torch.from_numpy(np.array(target_tensors))
    data = TensorDataset(feature_tensors,target_tensors)
    del feature_tensors
    del target_tensors
    data_loader = torch.utils.data.DataLoader(data,shuffle=False,batch_size = batchsize)
    return data_loader

In [5]:
stock_train_dataloader = batch_data_1(stock_feature_train_list,240,32,'f_ret_direction')
stock_test_dataloader = batch_data_1(stock_feature_test_list,240,32,'f_ret_direction')

In [6]:
class LSTM(nn.Module):
    
    def __init__(self, num_features,hidden_dim, n_layers, dropout=0.5,sequence_length=240):
        super(LSTM, self).__init__()
        # set class variables
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.num_features = num_features
        self.sequence_length = sequence_length
        # define model layers
        self.lstm = nn.LSTM(num_features,hidden_dim,n_layers,dropout = dropout, batch_first = True)
        self.fc = nn.Linear(hidden_dim*sequence_length,1)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x,hidden):
        x,hidden = self.lstm(x,hidden)
        #x = x.contiguous().view(-1, self.hidden_dim)
        x = x.contiguous().view(x.shape[0],-1)
        x = self.dropout(x)
        x = F.sigmoid(self.fc(x))
        x = x.squeeze()
        return x,hidden
    def init_hidden(self,batch_size):
        train_on_gpu=torch.cuda.is_available()
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [7]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden,clip=5):
    
    # move data to GPU, if available
    
    train_on_gpu=torch.cuda.is_available()
    if train_on_gpu:
        inp,target,rnn = inp.cuda(),target.cuda(),rnn.cuda()
    h = tuple([each.data for each in hidden])
    optimizer.zero_grad()
    
    output,h = rnn(inp,h)
    loss = criterion(output,target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()
    
    return loss.item(),h

In [8]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(stock_train_dataloader, 1):
            labels = labels.double()
            # make sure you iterate over completely full batches, only
            n_batches = len(stock_train_dataloader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

In [9]:
epoches = 10
num_features = 31
hidden_dim = 64
n_layers = 5
learning_rate = 0.00001
lstm = LSTM(num_features=num_features,hidden_dim=hidden_dim, n_layers = n_layers, dropout=0.5).double()
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    lstm.to("cuda")

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()
trained_lstm = train_rnn(lstm, 32, optimizer, criterion, epoches)

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 0.7136464314017259

Epoch:    1/10    Loss: 0.7150333337786173

Epoch:    1/10    Loss: 0.7089091453467894

Epoch:    2/10    Loss: 0.6965989980077001

Epoch:    2/10    Loss: 0.6957139544328319

Epoch:    2/10    Loss: 0.696077790848691

Epoch:    3/10    Loss: 0.6938060506870357

Epoch:    3/10    Loss: 0.6937593285556867

Epoch:    3/10    Loss: 0.6940169494219818

Epoch:    4/10    Loss: 0.6933448649939673

Epoch:    4/10    Loss: 0.6933349461129559

Epoch:    4/10    Loss: 0.6934573360309767

Epoch:    5/10    Loss: 0.6932272388984888

Epoch:    5/10    Loss: 0.6932199568105081

Epoch:    5/10    Loss: 0.6932780210473204

Epoch:    6/10    Loss: 0.6931823815910386

Epoch:    6/10    Loss: 0.6931780837405725

Epoch:    6/10    Loss: 0.6932051997025134

Epoch:    7/10    Loss: 0.6931647070508288

Epoch:    7/10    Loss: 0.6931609532852667

Epoch:    7/10    Loss: 0.6931770128653341

Epoch:    8/10    Loss: 0.6931565841441549

Epoch

In [ ]:
test_loss = 0
trained_lstm.eval()
hidden = trained_lstm.init_hidden(32)
for inputs,labels in stock_train_dataloader:
    train_on_gpu=torch.cuda.is_available()
    h = tuple([each.data for each in hidden])
    if train_on_gpu:
        inputs,trained_lstm,labels = inputs.cuda(),trained_lstm.cuda(),labels.cuda()
    output,h = trained_lstm(inputs,h)
    loss = criterion(output,labels)
    test_loss+=loss.item()
test_loss = test_loss/len(stock_test_dataloader.dataset)
print(test_loss)

In [ ]:
stock_train_dataloader_dict = {}
stock_test_dataloader_dict = {}
for batch_size in [32,64,128]:
  stock_train_dataloader_dict[batch_size] = batch_data_1(stock_feature_train_list,240,batch_size,'f_ret_1min')
  stock_test_dataloader_dict[batch_size] = batch_data_1(stock_feature_test_list,240,batch_size,'f_ret_1min')

In [ ]:
def grid_train_rnn(config,epoches = 20,checkpoint_dir = None):
    lstm_sabr = LSTM(31,config["hidden"],config["n_layer"],config["dropout"])
    lstm_sabr = lstm_sabr.double()
    train_on_gpu = torch.cuda.is_available()
    if train_on_gpu:
        lstm_sabr.cuda()
    optimizer = torch.optim.Adam(lstm.parameters(), lr=config["lr"])
    criterion = nn.MSELoss()
    train_loader = stock_train_dataloader_dict[config["batch_size"]]
    
    batch_losses = []  
    lstm_sabr.train()

    for epoch_i in range(1, epoches + 1):
        running_loss = 0
        # initialize hidden state
        hidden = lstm_sabr.init_hidden(config["batch_size"])
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
        # make sure you iterate over completely full batches, only
            n_batches = len(stock_train_dataloader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
          # forward, back prop
            loss, hidden = forward_back_prop(lstm_sabr, optimizer, criterion, inputs, labels, hidden)          
          # record loss
            batch_losses.append(loss)
            running_loss+=loss.item()

        with tune.checkpoint_dir(epoch_i) as checkpoint_dir:
            path = os.path.join(checkpoint_dir,"checkpoint")
            torch.save((lstm_sabr.state_dict(),optimizer.state_dict()),path)
        tune.report(loss = running_loss/len(train_loader.dataset))
    print("Finish Training")
